In [91]:
#Import necessary packages
import nltk
nltk.download('punkt')
from nltk.corpus import sentence_polarity
nltk.download('sentence_polarity')
import random
from nltk.tokenize import sent_tokenize
import re
import collections
from nltk.metrics import *
from nltk.classify import MaxentClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zeugn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package sentence_polarity to
[nltk_data]     C:\Users\zeugn\AppData\Roaming\nltk_data...
[nltk_data]   Package sentence_polarity is already up-to-date!


In [2]:
#Read text file that contains amazon product reviews
file = open('baby_text.txt', 'r')

In [3]:
# list of reviewText sentences
TextList = []
YearList = []

In [4]:
#reg ex to extract the review text and the year
review_pattern = r'''(?x) reviewText[:](.+) '''

year_pattern = r''' (?x) reviewTime[:](.+)'''

In [6]:
#Loop through the text file and extract the year as well as the review text
#Clean strings
for line in file:
    sentlist = []
    reviewyear_tokens = nltk.regexp_tokenize(line, year_pattern)
    reviewtext_tokens = nltk.regexp_tokenize(line, review_pattern)
        
    if(len(reviewtext_tokens)>0):
        current_review = reviewtext_tokens[0].strip("\n") 
        TextList.append(current_review)
    if(len(reviewyear_tokens)>0):
        current_year = reviewyear_tokens[0].split(',')[1].strip('\n').strip(' ')
        YearList.append(current_year)


In [7]:
# select a year to filter off and append review text      
final = []
for i in range(0, len(YearList)):
    if(YearList[i]=='2014'):
        final.append(TextList[i])
        

In [8]:
print("number of reviews",len(final))
final= final[:10000]


number of reviews 35691


In [9]:
print(len(final))

10000


In [10]:
#sentence tokenization on the review text
final_sentlist = []
for review in final:
    curr_sent_list = nltk.sent_tokenize(review)
    for sent in curr_sent_list:
        final_sentlist.append(sent)

In [11]:
final_sentlist[:5]


['Helps me know exactly how my babies day has gone with my mother in law watching him while I go to work.',
 'It also has a section for her to write notes and let me know anything she may need.',
 "I couldn't be happier with this book.",
 'I wanted an alternative to printing out daily log sheets for the nanny to fill out, and this has worked out great!',
 "I'm no longer searching my daughter's bag for a crumpled piece of paper each day."]

In [12]:
text_pattern2 = r''' (?x)
    [][.,;"'?!():-_%']    
'''

In [13]:
textList = []
for sent in final_sentlist:
    sent = sent.lower()
    words = nltk.word_tokenize(sent)
    if(len(words)==1):
        if(len(nltk.regexp_tokenize(words[0], text_pattern2))<0):
            textList.append(sent)
    else:
        textList.append(sent)

In [14]:
len(textList)

43821

In [15]:
final_sentlist = textList

In [16]:
len(final_sentlist)

43821

In [17]:
stopwords = nltk.corpus.stopwords.words('english')
bag_of_words = sentence_polarity.sents()
sentlabels = [(sent, label) for label in sentence_polarity.categories()
                     for sent in sentence_polarity.sents(categories=label)]


In [18]:
random.shuffle(sentlabels)

In [19]:
allsents = [word for (sent, label) in sentlabels for word in sent]
allsents_freq = nltk.FreqDist(allsents)
freqwords = allsents_freq.most_common(2000)
freqword_features = [word for (word,freq) in freqwords]

In [20]:
def document_features(sentence, words):
    unique_words = set(sentence)
    features = {}
    for word in words:
        features['contains({})'.format(word)] = (word in unique_words)
    return features

In [21]:

featureset = [(document_features(sent, freqword_features), label) for (sent, label) in sentlabels]

In [22]:

train_featureset, test_featureset = featureset[1000:], featureset[:1000]

In [23]:
classifier = nltk.NaiveBayesClassifier.train(train_featureset)

In [24]:
print("Baseline Accuracy",nltk.classify.accuracy(classifier, test_featureset))

Baseline Accuracy 0.76


In [25]:
def display_accuracy_measures(actual_set, input_classifier):
    reference_sets = collections.defaultdict(set)
    predicted_sets = collections.defaultdict(set)
    for i, (feats, label) in enumerate(actual_set):
        reference_sets[label].add(i)
        predicted = input_classifier.classify(feats)
        predicted_sets[predicted].add(i)
    print('Positive precision:', precision(reference_sets['pos'], predicted_sets['pos']))
    print('Positive recall:', recall(reference_sets['pos'], predicted_sets['pos']))
    print('Positive F-measure:', f_measure(reference_sets['pos'], predicted_sets['pos']))
    print('Negative precision:', precision(reference_sets['neg'], predicted_sets['neg']))
    print ('Negative recall:', recall(reference_sets['neg'], predicted_sets['neg']))
    print ('Negative F-measure:', f_measure(reference_sets['neg'], predicted_sets['neg'])) 

In [26]:
pos_sentencelist = []
neg_sentencelist = []

In [34]:
for sent in final_sentlist:
    if(classifier.classify(document_features(nltk.word_tokenize(sent),freqword_features)) == 'pos'):
        pos_sentencelist.append(sent)
    elif(classifier.classify(document_features(nltk.word_tokenize(sent),freqword_features)) == 'neg'):
        neg_sentencelist.append(sent)

In [35]:
print("Baseline Accuracy",nltk.classify.accuracy(classifier, test_featureset))

Baseline Accuracy 0.76


In [37]:
display_accuracy_measures(featureset, classifier)

Positive precision: 0.8034449760765551
Positive recall: 0.7874695179140874
Positive F-measure: 0.7953770367563471
Negative precision: 0.7916130218870701
Negative recall: 0.8073532170324517
Negative F-measure: 0.799405646359584


In [38]:
print("positive reviews ",len(pos_sentencelist))
print("negative reviews ",len(neg_sentencelist))

positive reviews  19842
negative reviews  47957


In [57]:
print(pos_sentencelist[:20])


['highly recommended as a perfect way to keep track and organize information in one easy to travel with place!', 'my son loves to chew on things and is still awaiting his first tooth.', 'it is a great toy for them to chew and play with, but it is a bit harder for smaller babies to chew on.', 'my daughter finds this very interesting especially when it gently vibrates and massages her gums.', 'this is a great teether.']


In [61]:
print(neg_sentencelist[6:12])

['this is great for basics, but i wish the space to write things in was bigger.', 'i wanted to love this, but it was pretty expensive for only a few months worth of calendar pages.', 'i ended up buying a regular weekly planner - 55% off!', '- the planner - that is 8 1/2 x 11 and has all seven days on the right page and the left page has room to write a to do list and goals.', 'this planner was cute, just not what i wanted.', 'has columns for all the info i need at a glance when i get home from work.']


In [62]:
negationwords = ['no', 'not', 'never', 'none', 'nowhere', 'nothing', 'rather', 'hardly', 'scarcely', 'rarely', 'neither', 'nor']

In [64]:
len(final_sentlist)

43821

In [65]:
def NOT_features(document, word_features, negationwords):
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = False
        features['contains(NOT{})'.format(word)] = False
    # Parse the document words in order
    for i in range(0, len(document)):
        word = document[i]
        if ((i + 1) < len(document)) and ((word in negationwords) or (word.endswith("n't"))):
            i += 1
            features['contains(NOT{})'.format(document[i])] = (document[i] in word_features)
        else:
            features['contains({})'.format(word)] = (word in word_features)
    return features

In [70]:
#Obtaining the feature set for the sentences 
feature_set_2 = [(NOT_features(sent, freqword_features, negationwords), label) for (sent, label) in sentlabels]

In [71]:
#Split the feature set data into training and test data
train_feature_set_2, test_feature_set_2 = feature_set_2[2000:], feature_set_2[:2000]

In [73]:
#Apply NB Classification on the training feature set
classifier_2 = nltk.NaiveBayesClassifier.train(train_feature_set_2)

In [76]:
#Determine the Accuracy
print("Negation Accuracy",nltk.classify.accuracy(classifier_2, test_feature_set_2))

Negation Accuracy 0.769


In [77]:
#Display the other measures
display_accuracy_measures(feature_set_2, classifier_2)

Positive precision: 0.9276278001148766
Positive recall: 0.9088351153629713
Positive F-measure: 0.9181353041500853
Negative precision: 0.9106453392167678
Negative recall: 0.9290939786156444
Negative F-measure: 0.9197771587743733


In [78]:
#Create the positive and negative lists
pos_sentence_list2 = []
neg_sentence_list2 = []

In [81]:
for sent in final_sentlist:
    if(classifier_2.classify(NOT_features(nltk.word_tokenize(sent),freqword_features, negationwords)) == 'pos'):
        pos_sentence_list2.append(sent)
    elif(classifier_2.classify(NOT_features(nltk.word_tokenize(sent),freqword_features,negationwords)) == 'neg'):
        neg_sentence_list2.append(sent)

In [82]:
#Printing the number of positive and negative reviews
print("The number of positive reviews ",len(pos_sentence_list2))
print("The number of negative reviews ",len(neg_sentence_list2))

The number of positive reviews  13777
The number of negative reviews  30044


In [86]:
print(pos_sentence_list2[6:10])


['i have used this tracker for all 3 of my children and swear by it!', 'i also used it to jot down other important &#34;mile stone&#34; things that i would have quickly forgotten&#8230;like outings, temperature readings, visits from friends to meet him, calls to make, when we stopped swaddling him, etc.', 'i went back to formally fill out his baby album since i would have forgotten so many of those valuable little tidbits of information & memories.', "we use this everyday to communicate feeding, medicine and other needs when i don't have time to give my nanny the full rundown."]


In [89]:
print(neg_sentence_list2[6:10])

["i would have preferred a plastic-type cover, but it's held up well so far.", 'this is great for basics, but i wish the space to write things in was bigger.', 'i wanted to love this, but it was pretty expensive for only a few months worth of calendar pages.', 'i ended up buying a regular weekly planner - 55% off!']
